# Introduction à la réduction de la dimensionnalité

Notre parcours dans la définition du modèle vectoriel nous a conduit·e de la constitution d’un sac de mots par divers procédés à l’établissement d’une matrice d’occurrences. Nous avons d’ailleur quitté le chapitre précédent avec un double problème : d’une part la seule approche fréquentielle était incapable de rendre compte de l’importance relative d’un terme dans un document et encore moins dans un corpus ; d’autre part la constitution d’une matrice d’occurrences nous avait rapidement amené·e, en raison de la taille du vocabulaire commun à considérer, à évoluer dans des espaces en très haute dimension qui sont majoritairement remplis de vide.

Dans ce chapitre, nous allons remédier au premier et dessiner l’ébauche d’une méthode pour projeter les documents dans un espace réduit.

## Évaluer l’importance d’un terme dans un document (TF-IDF)

Nous l’avons vu précédemment, certains termes dans un corpus sont plus importants que d’autres pour caractériser un texte par rapport à un autre, et leur importance n’est souvent pas proportionnelle à leur fréquence. De là découle la nécessité de repérer les mots vides de sens, les *stopwords*, pour les retirer du sac de mots (BoW) qui le représente. Pour autant, la méthode BoW se contente d’une mesure fréquentielle sans établir de rapport d’importance entre les termes. La matrice d’occurrences reste ainsi assez pauvre pour rendre compte de sémantique.

Une autre approche, largement répandue dans le traitement automatique du langage naturel, parvient à inférer, de l’analyse fréquentielle, une certaine valeur d’importance aux termes contenus dans le sac de mots. Cette approche repose sur deux principes : la fréquence du terme (TF) et la fréquence du terme dans le corpus (IDF) qui prêtent une signification à la rareté d’un terme.

Sans parler de robustesse, la justification de cette approche repose sur la loi de Zipf qui prévoit que la fréquence d’un terme dans un texte est liée à son rang dans l’ordre des fréquences : le mot le plus fréquent apparaîtrait dix fois plus souvent que le dixième mot le plus fréquent, cent fois plus que le centième etc. En grande partie pour cette raison, la méthode TF-IDF ne souffre pas de la présence des mots vides dans le sac de mots.

Dans sa formulation générale, l’expression mathématique vaut :

$$
\text{TF-IDF}(t, d) = \text{TF}(t, d) \cdot \ln\left(\frac{N}{\text{df}(t)}\right)
$$

Où :

- *t* est le token ;
- *d* figure le nombre total de tokens dans le document ;
- *N* représente le nombre total de documents ;
- et $\text{df}(t)$ calcule la fréquence du token dans le document.

On lui préfère toutefois souvent la version lissée afin d’éviter des divisions par zéro et de privilégier des mots très rares qui pourraient avoir une empreinte trop importante :

$$
\text{TF-IDF}(t, d) = \text{TF}(t, d) \cdot \ln\left(\frac{N}{1 + \text{df}(t)}\right)
$$

### La fréquence du terme (TF)

De l’anglais *term frequency*, la fréquence du terme établit un rapport entre le nombre d’occurrences d’un token ($t$) dans un document et le nombre total de mots dans ce document ($d$) :

$$
\text{TF}(t, d) = \frac{t}{d}
$$

Prenons un corpus constitué de trois textes :

```txt
(A) Le petit chat boit du lait. Le lait n’est pas bon pour les chats.
(B) Les petits chiens boivent de l’eau. L’eau irait aussi aux chats.
(C) À partir d’un moment, eau ou lait, ils peuvent bien boire ce qu’ils veulent.
```

La taille en mots du document *A* est de 15, quand elle est de 13 pour le document *B* et de 16 pour le document *C*. Intéressons-nous au mot *lait* ($1$) qui apparaît deux fois dans *A* et une fois dans *C*, mais jamais dans *B*. Ses fréquences seront :

$$
\begin{align*}
    \text{TF}_{1, A} &= \frac{2}{15} = 0,1333 \\
    \text{TF}_{1, B} &= \frac{0}{13} = 0 \\
    \text{TF}_{1, C} &= \frac{1}{16} = 0,0625 \\
\end{align*}
$$

### La fréquence inverse de document (IDF)

Quand la mesure TF s’attachait au terme dans un document, la mesure IDF (*inverse document frequency*) va s’intéresser à la présence du terme dans le corpus entier selon la relation suivante où $d$ représente le nombre de documents où le terme apparaît et $N$ le nombre total de documents :

$$
\text{IDF}(t) = \ln{\frac{N}{\text{df}(t)}}
$$

Dans notre exemple, la mesure IDF pour le mot *lait* vaut :

$$
\text{IDF}_1 = \ln{\frac{3}{2}} \approx 0.4055
$$

Le calcul du logarithme permet de pondérer le rapport entre $N$ et $d$ dans la mesure où, lors de l’obtention de TF, le résultat était situé dans un intervalle $[0, 1]$.

### La mesure TF-IDF

Au final, la formule TF-IDF est un produit entre TF et IDF. Pour notre exemple avec le mot *lait* :

$$
\begin{align*}
    \text{TF-IDF}_{1, A} &= \frac{2}{15} \cdot ln \frac{3}{2} \approx 0,0541 \\
    \text{TF-IDF}_{1, B} &= \frac{0}{13} \cdot ln \frac{3}{2} = 0 \\
    \text{TF-IDF}_{1, C} &= \frac{1}{16} \cdot ln \frac{3}{2} \approx 0,0253 \\
\end{align*}
$$

Au regard du mot *lait*, le document *A* apparaît ainsi comme plus pertinent. Notons que si nous avions opté pour le TF-IDF avec lissage, nous aurions obtenu des scores de 0.

### Limitations du modèle

Quoique très largement utilisé pour sa facilité de mise en œuvre en dépit du coût en termes de calcul machine, la mesure TF-IDF souffre principalement de son incapacité à traiter la sémantique du terme.

Gardons par ailleurs à l’esprit que la formule accordera mécaniquement davantage d’importance aux documents très volumineux, aussi faudra-t-il penser à les pénaliser ou bien à ne travailler que sur des corpus équilibrés.

### Normalisation des pondérations

Pour atténuer l’effet des documents de tailles différentes qui auront des valeurs plus élevées, les scores TF-IDF sont souvent normalisés. Deux normes sont retenues :

1. La norme $L^1$ qui divise chaque pondération par la somme absolue des pondérations ;
2. La norme $L^2$ qui divise chaque pondération TF-IDF par la norme euclidienne du vecteur.

#### Norme $L^1$

La formule vaut :

$$
\text{Norme } L^1 : \frac{x_i}{\sum |x_i|}
$$

Si nous l’appliquons au mot *lait*, nous calculons la somme absolue des pondérations :

$$
\sum |x_i| = 0.0541 + 0 + 0.0253 = 0.0794
$$

Avant de diviser chaque valeur avec :

$$
\begin{align*}
    \text{TFIDF}_{1, A} &= \frac{0.0541}{0.0794} = 0.6814 \\
    \text{TFIDF}_{1, B} &= \frac{0}{0.0794} = 0 \\
    \text{TFIDF}_{1, C} &= \frac{0.0253}{0.0794} = 0.3186 \\
\end{align*}
$$

Cette fois-ci, l’addition des pondérations sera toujours égale à 1.

#### Norme $L^2$

Il s’agit de la formule retenue par défaut par les algorithmes des bibliothèques spécialisées et s’exprime par :

$$
\text{Norme } L^2 : \frac{x_i}{\sqrt{\sum x_i^2}}
$$

Si nous calculons d’abord le dénominateur :

$$
\sqrt{\sum x_i^2} = \sqrt{(0.0541)^2 + (0)^2 + (0.0253)^2} = \sqrt{0.002927 + 0 + 0.000640} = 0.0597
$$

Nous pouvons ensuite normaliser les pondérations du mot *lait* :

$$
\begin{align*}
    \text{TFIDF}_{1, A} &= \frac{0.0541}{0.0597} = 0.9062 \\
    \text{TFIDF}_{1, B} &= \frac{0}{0.0597} = 0 \\
    \text{TFIDF}_{1, C} &= \frac{0.0253}{0.0597} = 0.4238 \\
\end{align*}
$$

Si avec la norme $L^2$ la somme des pondérations n’est plus égale à 1, la norme du vecteur $\vec{\text{lait}}$ l’est toutefois :

$$
\| \vec{\text{lait}} \| = \sqrt{(0.9061)^2 + (0)^2 + (0.4238)^2} = \sqrt{0.821 + 0 + 0.179} = \sqrt{1} = 1
$$

## Réduire la dimensionnalité

Avec la matrice de pondération TF-IDF normalisée selon $L^1$ ou $L^2$, nous avons réintroduit une représentation vectorielle plus équilibrée des documents. Si ces méthodes permettent de mieux comparer les documents entre eux et de capturer des relations plus subtiles entre les termes, la représentation reste de haute dimension, ce qui peut poser des défis en termes de calculs et d’interprétabilité.

L’esprit humain n’est pas habitué à se représenter des figures géométriques au-delà de trois dimensions – à part peut-être pour l’octachore, un 4-cube, popularisé sous la dénomination de tesseract, aussi est-il souvent nécessaire d’adopter une stratégie pour réduire leur dimensionnalité. En changeant de perspective, un cube peut se réduire à un carré, un carré à une ligne, une ligne à un point.

Grâce à des techniques comme **l’analyse sémantique latente** (*Latent Semantic Analysis* en anglais, ou LSA), nous pouvons projeter les vecteurs représentant les documents dans un sous-espace où des structures sémantiques latentes, jusque-là invisibles, émergent. Cette transformation nous permet non seulement de rendre les calculs plus efficaces, mais aussi de révéler des relations sémantiques profondes entre les documents, facilitant ainsi des tâches comme la recherche d’information, la classification, ou le clustering.

![Hypercube 0D to 4D](./images/hypercube_0D-4D.gif)

Vitaly Ostrosablin. – *From Point to Tesseract*. – CC BY-SA 3.0 &lt;https://creativecommons.org/licenses/by-sa/3.0&gt;, via Wikimedia Commons

### Le fléau de la dimension

À l’heure actuelle, notre document, le texte de *Salammbô*, est représenté par une matrice de dimension $ \times 5798$, soit bien plus qu’un tesseract – et on a vu le grabuge que ça a fini par causer aux Avengers ! Épargnons-leur des ennuis à venir et optons pour la réduction de nos données à une dimension moindre. D’autres bénéfices bien réels sont attendus :
- une visualisation facilitée des données ;
- la réduction des coûts de calculs ou d’acquisition ;
- l’amélioration de la qualité de modèles d’apprentissage.

Une fois les avantages énumérés, qu’avons-nous réellement dit du problème ? Pourquoi la dimensionnalité serait-elle un fléau ? L’expression *Curse of dimensionality* vient du mathématicien Richard Ernest Bellman pour qualifier entre autres le coût de résolution d’un problème pour une variable supplémentaire (Richard Ernest Bellman, *Adaptive control processes: a guided tour*, Princeton University Press, 1961). Un statisticien américain, Leo Breiman, fournit un exemple de comparaison. Considérons une matrice de dimension $100 \times 1$ avec des valeurs comprises dans l’intervalle $[0,1]$ :

|Observation|Valeur|
|-|-|
|$n_1$|0.1891|
|$n_2$|0.7023|
|$n_3$|0.0452|
|$n_i$|…|
|$n_{100}$|0.9036|

Ces observations occupent une certaine place dans l’espace unidimensionnel et il est assez simple d’en obtenir une visualisation ou d’inférer des résultats. Si maintenant nous envisageons un espace à seulement dix dimensions, il faudrait $10^{20}$ observations pour obtenir une densité équivalente et permettre une analyse statistique comparable.

Autre exemple peut-être un peu plus parlant : la distance moyenne entre deux points pris au hasard dans un carré de 1 cm de côtés (un hypercube 2D unité) sera de 0,52 cm alors qu’elle sera de 0,66 cm dans un cube ! Pour aller plus loin, la distance moyenne entre deux points sera de 4 cm dans un hypercube unité à 10 dimensions, de 12 cm dans un hypercube unité à 1000 dimensions etc. En somme, les espaces de grande dimension sont majoritairement constitués de vide.

Un autre fait amusant est que dans un carré unité, il est très improbable qu’un point tiré au hasard soit proche d’un bord. Les chances d’être à moins de 0,001 cm d’un côté sont à peu près de 0,4 % dans un carré de 1 cm. Et dans les espaces de grande dimension, c’est l’inverse ! Autrement dit, plus on multiplie les variables, plus il y a de risques de rencontrer des valeurs extrêmes dans une dimension !

Pour finir de se convaincre, prenons une dizaine d’observations décrites par deux variables :

|Observation|Variable 1|Variable 2|
|-|-|-|
|$n_1$|1,2|6,3|
|$n_2$|4,3|5|
|$n_3$|3,5|4|
|$n_4$|1|4|
|$n_5$|2.3|5|
|$n_6$|4|9.7|
|$n_7$|6.8|5.4|
|$n_8$|8.1|9|
|$n_9$|6|1|
|$n_{10}$|5|5|

Et projetons-les dans un plan :

![Projection 2D](./images/2D-projection.png)

Le cercle en orange délimite une frontière d’un rayon de 2 unités autour de l’observation $n_{10}$ afin de montrer que ses plus proches voisins sont les observations $n_2$, $n_3$ et $n_7$.

Si maintenant nous projetons les mêmes données dans un espace unidimensionnel en considérant que la variable 2 est inutile pour les décrire, nous voyons que les plus proches voisins ne $n_{10}$ sont maintenant $n_2$, $n_6$ et $n_9$ :

![Projection 1D](./images/1D-projection.png)

### Méthodes de réduction

La réduction de dimension repose sur l’hypothèse, très souvent vérfiée, selon laquelle des données dans un espace *n*-dimensionnel sont situées à proximité d’un sous-espace *k*-dimensionnel. Quel que soit l’algorithme de réduction envisagé, il implique nécessairement une perte d’information qu’il faudra minimiser en le paramétrant justement.

#### L’approche naïve

Dans un projet, en fonction de l’objet d’étude, il ne sera pas nécessaire de conserver toutes les variables qui décrivent les observations. Si l’on veut étudier la relation entre la taille et le poids d’un individu, il sera sans doute intéressant de connaître son âge, son sexe, son régime alimentaire, son activité physique, mais inutile de retenir la marque de sa voiture, le nombre de voyages qu’il a effectués ou s’il a fréquenté un établissement hospitalier dans les cinq dernières années.

L’apport de la littérature scientifique ou de l’avis d’un·e expert·e du domaine pourront être d’une aide précieuse.

#### Révéler des structures sous-jacentes

L’idée principale des méthodes de réduction de la dimensionnalité est de révéler des structures latentes reliant les dimensions, alors que la majorité d’entre elles ne contribue qu’à une infime partie aux phénomènes étudiés. Pour y parvenir, ces méthodes s’appuient sur des opérations fondamentales et robustes de l’algèbre linéaire (produit matriciel, produit extérieur, décomposition matricielle, etc.) afin de manipuler les données et faire émerger des schémas significatifs, comme des variances inter et intra-clusters ou encore des relations sémantiques.

Parmi la multitude de méthodes, nous pouvons citer les suivantes :

- **L’analyse en composantes principales (PCA) :** Une méthode non supervisée qui cherche à maximiser la variance expliquée. Elle est basée sur la décomposition en valeurs singulières (SVD) ou sur la diagonalisation d’une matrice de covariance pour identifier les axes principaux des données.
- **La factorisation de matrices non-négatives (NMF) :** Une méthode non supervisée qui approxime une matrice par le produit de deux matrices de rang inférieur avec des éléments non négatifs. Elle est souvent utilisée pour extraire des composantes significatives comme des sujets dans des documents.
- **L’analyse discriminante linéaire (LDA) :** Utilisée principalement pour la classification, cette méthode supervisée cherche à maximiser la séparation entre les classes tout en minimisant la variance intra-classe.
- **L’analyse sémantique latente (LSA) :** En utilisant la décomposition SVD sur une matrice termes-documents, la LSA parvient à capturer des relations entre les termes.
- **L’allocation de Dirichlet latente :** Une méthode probabiliste qui identifie des thèmes latents dans des documents en représentant chaque document comme une distribution de sujets et chaque sujet comme une distribution de mots.
- **La t-SNE :** Une méthode probabiliste non linéaire qui réduit la dimensionnalité tout en préservant les relations locales, utile pour visualiser des données complexes dans deux ou trois dimensions.

Chacune a ses avantages, ses limites et des contextes d’application spécifiques qui impliquent de les choisir avec soin.